In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys

radar_path = "/content/drive/MyDrive/Radar"
if radar_path not in sys.path:
    sys.path.insert(0, radar_path)
data_dir = "data-100"

In [5]:
!pip install scipy numpy matplotlib

In [6]:
import math
import numpy as np
from scipy import signal


def generateNoisyData(f_s, sample_length, SNR, offset=False):
    """
    Create a discrete series of random noise measurements, simulating no drone signal.
    :param f_s: the sampling frequency in Hz
    :param sample_length: the length of the sample in seconds
    :returns: a tuple of time values x, and their corresponding noisy values.
    """
    xs = []
    noisy_ys = []
    A_r = np.random.chisquare(4)
    variance = 10**(2*math.log10(A_r) - (SNR/10))
    for i in range(int(f_s * sample_length)):
        x = i / f_s
        xs.append(x)
        # Generating random noise without incorporating drone signal
        noisy_val = np.random.normal(0, math.sqrt(variance))
        noisy_ys.append(noisy_val)

    return xs, noisy_ys


def generateNoisySTFT(f_s, sample_length, SNR, offset=False):
    xs, noisy_ys = generateNoisyData(f_s, sample_length, SNR)

    # Perform a Fourier transform on the noisy signal
    f, t, Zreal = signal.stft(
        noisy_ys, f_s, window='hamming', nperseg=16, noverlap=8, return_onesided=False
    )
    Xreal = 20 * np.log10(np.abs(np.fft.fftshift(Zreal, axes=0)))

    xs, noisy_ys = generateNoisyData(f_s, sample_length, SNR)

    f, t, Zimag = signal.stft(
        noisy_ys, f_s, window='hamming', nperseg=16, noverlap=8, return_onesided=False)

    Ximag = 20*np.log10(np.abs(np.fft.fftshift(Zimag, axes=0)))

    return np.stack((Xreal, Ximag))

In [7]:
import os
from configurations import *
from fourier import plotSTFT
from scipy import signal
import numpy as np
from drone_constants import drones

sample_length = 0.20 # in seconds
Xf_s = 10_000  # sample frequency for X band, in hz

testset_size = 20  # per denomination
trainset_size = 100  # per denomination

# config = dict(scenarioWband, **djimatrice300rtk)
sampling_frequencies = [Xf_s]
SNR = [10, 5, 0, -5]

noisy_STFT = generateNoisySTFT(Xf_s, sample_length, SNR[0])

print(noisy_STFT.shape)


(2, 16, 251)


In [9]:
for f_s in sampling_frequencies:
    for SNR in SNR:
        for drone in drones:
            scenario = scenarioXband

            print(SNR)
            print(drone["name"])
            print(f_s)
            scenario["SNR"] = SNR
            # make the directory
            os.system(f"mkdir -p {radar_path}/{data_dir}/testset/{f_s}fs/{SNR}SNR/noise")

            print(f"Generating test data for {f_s=}, {SNR=}")
            for x in range(testset_size):
                # new signal function every time

                noisy_STFT = generateNoisySTFT(Xf_s, sample_length, SNR)

                np.save(f"{radar_path}/{data_dir}/testset/{f_s}fs/{SNR}SNR/noise/{x:06}.npy", noisy_STFT)

            os.system(f"mkdir -p {radar_path}/{data_dir}/trainset/{f_s}fs/{SNR}SNR/noise")
            print(f"Generating training data for {f_s=}, {SNR=}")
            for x in range(trainset_size):

                noisy_STFT = generateNoisySTFT(Xf_s, sample_length, SNR)


                np.save(f"{radar_path}/{data_dir}/trainset/{f_s}fs/{SNR}SNR/noise/{x:06}.npy", noisy_STFT)


10
DJI_Matrice_300_RTK
10000
Generating test data for f_s=10000, SNR=10
Generating training data for f_s=10000, SNR=10
10
DJI_Mavic_Air_2
10000
Generating test data for f_s=10000, SNR=10
Generating training data for f_s=10000, SNR=10
10
DJI_Mavic_Mini
10000
Generating test data for f_s=10000, SNR=10
Generating training data for f_s=10000, SNR=10
10
DJI_Phantom_4
10000
Generating test data for f_s=10000, SNR=10
Generating training data for f_s=10000, SNR=10
10
Parrot_Disco
10000
Generating test data for f_s=10000, SNR=10
Generating training data for f_s=10000, SNR=10
5
DJI_Matrice_300_RTK
10000
Generating test data for f_s=10000, SNR=5
Generating training data for f_s=10000, SNR=5
5
DJI_Mavic_Air_2
10000
Generating test data for f_s=10000, SNR=5
Generating training data for f_s=10000, SNR=5
5
DJI_Mavic_Mini
10000
Generating test data for f_s=10000, SNR=5
Generating training data for f_s=10000, SNR=5
5
DJI_Phantom_4
10000
Generating test data for f_s=10000, SNR=5
Generating training data